# Tensorflow for R 
## 1. Basic Example
(source : https://tensorflow.rstudio.com)

Here’s a simple example of making up some data in two dimensions and then fitting a line to it:

In [10]:
#To get started, install the tensorflow R package from CRAN as follows:
library(tensorflow)

In [17]:
# Then, use the install_tensorflow() function to install TensorFlow
install_tensorflow()

Using r-tensorflow conda environment for TensorFlow installation

Installation complete.



In [22]:
# You can confirm that the installation succeeded with:
sess  = tf$Session()
hello = tf$constant('Hello, Tensorflow~')
sess$run(hello)

b'Hello, Tensorflow~'

In [23]:
# Create 100 phony x, y data points, y = x * 0.1 + 0.3
x_data = runif(100, min=0, max=1)
y_data = x_data * 0.1 + 0.3

In [24]:
# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)

W = tf$Variable(tf$random_uniform(shape(1L), -1.0, 1.0))
b = tf$Variable(tf$zeros(shape(1L)))
y = W * x_data + b

In [25]:
# Minimize the mean squared errors.
loss      = tf$reduce_mean((y - y_data)^2)
optimizer = tf$train$GradientDescentOptimizer(0.5)
train     = optimizer$minimize(loss)

In [26]:
# Launch the graph and initialize the variables.
sess      = tf$Session()
sess$run(tf$global_variables_initializer())

In [27]:
# Fit the line (Learns best fit is W: 0.1, b: 0.3)
for (step in 1:201) {
    sess$run(train)
    if(step %% 20 == 0)
        cat(step, '-', sess$run(W), sess$run(b), '\n')
}

20 - 0.2181985 0.2398266 
40 - 0.1311855 0.2841238 
60 - 0.108228 0.2958112 
80 - 0.1021709 0.2988948 
100 - 0.1005728 0.2997084 
120 - 0.1001511 0.2999231 
140 - 0.1000399 0.2999797 
160 - 0.1000105 0.2999946 
180 - 0.1000028 0.2999986 
200 - 0.1000007 0.2999996 


## 2. MNIST Example

### The MNIST Data

The MNIST data is hosted on Yann LeCun’s website. If you are copying and pasting in the code from this tutorial, start here with these three lines of code which will download and read in the data automatically:
- The MNIST data is split into three parts: 55,000 data points of training data (mnist\$train), 10,000 points of test data (mnist\$test), and 5,000 points of validation data (mnist\$validation)
- As mentioned earlier, every MNIST data point has two parts: an image of a handwritten digit and a corresponding label. We’ll call the images “x” and the labels “y”. Both the training set and test set contain images and their corresponding labels; for example the training images are mnist\$train\$images and the training labels are mnist\$train\$labels.
- Each image is 28 pixels by 28 pixels. We can interpret this as a big array of numbers:
![MNIST-Matrix](MNIST-Matrix.png)
- We can flatten this array into a vector of 28x28 = 784 numbers. It doesn’t matter how we flatten the array, as long as we’re consistent between images. From this perspective, the MNIST images are just a bunch of points in a 784-dimensional vector space, with a very rich structure (warning: computationally intensive visualizations).


- The result is that mnist\$train\$images is a tensor (an n-dimensional array) with shape (55000L, 784L). The first dimension is an index into the list of images and the second dimension is the index for each pixel in each image. Each entry in the tensor is a pixel intensity between 0 and 1, for a particular pixel in a particular image.
![mnist-train-xs](mnist-train-xs.png)
- Each image in MNIST has a corresponding label, a number between 0 and 9 representing the digit drawn in the image.
- For the purposes of this tutorial, we’re going to want our labels as “one-hot vectors”. A one-hot vector is a vector which is 0 in most dimensions, and 1 in a single dimension. In this case, the n th digit will be represented as a vector which is 1 in the n th dimension. For example, 3 would be [0,0,0,1,0,0,0,0,0,0] Consequently, mnist\$train\$labels is a (55000L, 10L) array of floats.

### Softmax Regressions

A softmax regression has two steps: first we add up the evidence of our input being in certain classes, and then we convert that evidence into probabilities.
 - To tally up the evidence that a given image is in a particular class, we do a weighted sum of the pixel intensities. The weight is negative if that pixel having a high intensity is evidence against the image being in that class, and positive if it is evidence in favor.
 - The following diagram shows the weights one model learned for each of these classes. Red represents negative weights, while blue represents positive weights.
 ![softmax](softmax-weights.png)
 
 We also add some extra evidence called a bias. Basically, we want to be able to say that some things are more likely independent of the input. The result is that the evidence for a class 
i
 given an input 
x
 is:
<div align="center"> $\text{evidence}_i = \sum_j W_{i,~ j} x_j + b_i$ </div>

where 
W
i
 is the weights and 
b
i
 is the bias for class 
i
, and 
j
 is an index for summing over the pixels in our input image 
x
. We then convert the evidence tallies into our predicted probabilities 
y
 using the “softmax” function:
 
 <div align='center'> $y = \text{softmax}(\text{evidence})$ </div>
 
 Here softmax is serving as an “activation” or “link” function, shaping the output of our linear function into the form we want – in this case, a probability distribution over 10 cases. You can think of it as converting tallies of evidence into probabilities of our input being in each class. It’s defined as:
 <div align='center'> $\text{softmax}(x) = \text{normalize}(\exp(x))$ </div>
 
 If you expand that equation out, you get:
 <div align='center'> $\text{softmax}(x)_i = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$ </div>
 
- You can picture our softmax regression as looking something like the following, although with a lot more 
x
s. For each output, we compute a weighted sum of the 
x
s, add a bias, and then apply softmax.
![softmax-regression-scalargraph](softmax-regression-scalargraph.png)


- If we write that out as equations, we get:
![softmax-regression-scalarequation](softmax-regression-scalarequation.png)


- We can “vectorize” this procedure, turning it into a matrix multiplication and vector addition. This is helpful for computational efficiency. (It’s also a useful way to think.)
![softmax-regression-vectorequation](softmax-regression-vectorequation.png)


- More compactly, we can just write:
<div align='center'> $y = \text{softmax}(Wx + b)$ </div>

### Implementing the Regression

TensorFlow does its heavy lifting outside R, but it takes things a step further to avoid this overhead. Instead of running a single expensive operation independently from R, TensorFlow lets us describe a graph of interacting operations that run entirely outside R (Approaches like this can be seen in a few machine learning libraries.)

In [38]:
library(tensorflow)
datasets = tf$contrib$learn$datasets
mnist    = datasets$mnist$read_data_sets('MNIST-data', one_hot=T)

x isn’t a specific value. It’s a placeholder, a value that we’ll input when we ask TensorFlow to run a computation. 
- We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point numbers, with shape (NULL, 784L). (Here NULL means that a dimension can be of any length.)

We also need the weights and biases for our model. We could imagine treating these like additional inputs, but TensorFlow has an even better way to handle it: Variable. 
- A Variable is a modifiable tensor that lives in TensorFlow’s graph of interacting operations. It can be used and even modified by the computation. For machine learning applications, one generally has the model parameters be Variables.

In [39]:
x = tf$placeholder(tf$float32, shape(NULL, 784L))

W = tf$Variable(tf$zeros(shape(784L, 10L)))
b = tf$Variable(tf$zeros(shape(10L)))

> We create these Variables by giving tf$Variable the initial value of the Variable: 
- in this case, we initialize both W and b as tensors full of zeros. Since we are going to learn W and b, it doesn’t matter very much what they initially are.

> Notice that W has shape (784, 10) because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. b has shape (10) so we can add it to the output.

We can now implement our model. It only takes one line to define it!

In [40]:
y = tf$nn$softmax(tf$matmul(x, W) + b)

> First, we multiply x by W with the expression tf\$matmul(x, W). This is flipped from when we multiplied them in our equation, where we had 
W
x
, as a ***small trick to deal with x being a 2D tensor with multiple inputs***. We then add b, and finally apply tf\$nn\$softmax.

### Training the Model

In order to train our model, we need to define what it means for the model to be good. Well, actually, in machine learning we typically define what it means for a model to be bad. We call this the cost, or the loss, and it represents how far off our model is from our desired outcome. We try to minimize that error, and the smaller the error margin, the better our model is.
- One very common, very nice function to determine the loss of a model is called “cross-entropy.” 
<div align='center'>$H_{y'}(y) = -\sum_i y'_i \log(y_i)$</div>
> Where 
y
 is our predicted probability distribution, and 
y
′
 is the true distribution (the one-hot vector with the digit labels). In some rough sense, the cross-entropy is measuring how inefficient our predictions are for describing the truth

To implement cross-entropy we need to first add a new placeholder to input the correct answers:

In [42]:
y_ = tf$placeholder(tf$float32, shape(NULL, 10L))

Then we can implement the cross-entropy function

In [43]:
cross_entropy = tf$reduce_mean(-tf$reduce_sum(y_ * tf$log(y), 
                               reduction_indices=1L))

> - First, tf\$log computes the logarithm of each element of y. 
- Next, we multiply each element of y_ with the corresponding element of tf\$log(y). 
- Then tf\$reduce_sum adds the elements in the second dimension of y, due to the reduction_indices=1L parameter. 
- Finally, tf\$reduce_mean computes the mean over all the examples in the batch.
- Note that tensor indices (like the one used for reduction_indices) are 0-based within the TensorFlow API (rather than 1-based as is typical with R vectors).

Now that we know what we want our model to do, it’s very easy to have TensorFlow train it to do so. Because TensorFlow knows the entire graph of your computations, it can automatically use the backpropagation algorithm to efficiently determine how your variables affect the loss you ask it to minimize. Then it can apply your choice of optimization algorithm to modify the variables and reduce the loss.

In [44]:
optimizer  = tf$train$GradientDescentOptimizer(0.5)
train_step = optimizer$minimize(cross_entropy)

> In this case, we ask TensorFlow to minimize cross_entropy using the gradient descent algorithm with a learning rate of 0.5. Gradient descent is a simple procedure, where TensorFlow simply shifts each variable a little bit in the direction that reduces the cost. But TensorFlow also provides [many other optimization algorithms] 

> What TensorFlow actually does here, behind the scenes, is to add new operations to your graph which implement backpropagation and gradient descent. Then it gives you back a single operation which, when run, does a step of gradient descent training, slightly tweaking your variables to reduce the loss.

Now we have our model set up to train. One last thing before we launch it, we have to create an operation to initialize the variables we created. Note that this defines the operation but does not run it yet:

In [46]:
init = tf$global_variables_initializer()

We can now launch the model in a Session, and now we run the operation that initializes the variables:

In [47]:
sess = tf$Session()
sess$run(init)

Let’s train – we’ll run the training step 1000 times!

In [52]:
for(i in 1:1000){
    batches  = mnist$train$next_batch(100L)
    batch_xs = batches[[1]]
    batch_ys = batches[[2]]
    sess$run(train_step, feed_dict = dict(x = batch_xs, y_ = batch_ys))
}

> Each step of the loop, we get a “batch” of one hundred random data points from our training set. We run train_step feeding in the batches data to replace the placeholders.

### Evaluating Our Model

Well, first let’s figure out where we predicted the correct label. tf\$argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis. For example, tf\$argmax(y, 1L) is the label our model thinks is most likely for each input, while tf\$argmax(y_, 1L) is the correct label. We can use tf$equal to check if our prediction matches the truth.

In [59]:
correct_prediction = tf$equal(tf$argmax(y, 1L), tf$argmax(y_, 1L))

> Note that since tensors in the TensorFlow API are 0-based we pass 1L to specify that tf$argmax should operate on the second dimension of the tensor.

> correct_prediction is a vector of booleans. To determine what fraction are correct, we cast it to floating point numbers and then take the mean. For example, (TRUE, FALSE, TRUE, TRUE) would become (1.0, 0.0, 1.0, 1.0) which would become 0.75.

In [60]:
accuracy = tf$reduce_mean(tf$cast(correct_prediction, tf$float32))

Finally, we ask for our accuracy on our test data.

In [61]:
sess$run(accuracy, feed_dict = dict(x = mnist$test$images, 
                                    y_ = mnist$test$labels))

[1] 0.9188

> This should be about 92%